In [ ]:
from quantopian.algorithm import order_optimal_portfolio
from quantopian.algorithm import attach_pipeline, pipeline_output, order_optimal_portfolio
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.filters import QTradableStocksUS
import quantopian.optimize as opt
from quantopian.optimize import TargetWeights
import quantopian.algorithm as algo
from quantopian.pipeline.experimental import risk_loading_pipeline

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer, StandardScaler

In [ ]:
def initialize(context):
    
    # Rebalance every day, 1 hour after market open.
    algo.schedule_function(
        rebalance,
        #build_model,
        algo.date_rules.every_day(),
        algo.time_rules.market_open(hours=1)
    )
    context.security = [sid(24), sid(5061), sid(8554)]
    context.model = RandomForestClassifier(max_depth=3, random_state=100, n_estimators=15)
 
    # Use price history for the last 3 days
    context.lookback = 3  
    # Consider data for past two years or so
    context.history_range = 502  
    
    # Set comission and slippage rules
    set_slippage(slippage.FixedSlippage(spread=0.25))
    set_commission(commission.PerShare(0.15))


    # contrains to the contest
    context.max_posTam = 0.025
    context.max_lever = 0.98
    context.max_Volum = 0.85

    # Create our dynamic stock selector.
    algo.attach_pipeline(risk_loading_pipeline(), 'risk_loading_pipeline')
    algo.attach_pipeline(make_pipeline(), 'pipeline')

In [ ]:
def before_trading_start(context, data):
    # Obtiene la salida del pipeline y la guarda en context
    
    context.output = pipeline_output('pipeline')
    context.risk_factor_betas = pipeline_output('risk_loading_pipeline')

In [ ]:
def make_pipeline():
   
    # Base universe set to the QTradableStocksUS
    base_universe = QTradableStocksUS()
    
    
    sentiment_score = SimpleMovingAverage(
        inputs=[stocktwits.bull_minus_bear],
        window_length=5,
        mask=QTradableStocksUS()
    )
    close_price = USEquityPricing.close.latest
    return Pipeline(
        columns={
            'close_price':close_price,
            'sentiment_score': sentiment_score,
        },
        screen=sentiment_score.notnull()
    )
    
    return pipe

In [ ]:
def build_model(context, data):
    # Get daily prices
    recent_prices = data.history(context.security, 'price', context.history_range, '1d')
    # Get price changes
    price_changes = np.diff(recent_prices) > 0
 
    X = []  # Independent/input variables
    Y = []  # Dependent/output variable
    
    # Per day in our history
    for i in range(context.history_range-context.lookback-1):
        # Store prior price changes
        X.append(price_changes[i:i+context.lookback]) 
        # Store the day's price change
        Y.append(price_changes[i+context.lookback])
 
    # Create model
    context.model.fit(X, Y) 

In [ ]:
def rebalance(context, data):
    
    pipeline_output_df = pipeline_output('pipeline').dropna(how='any')
    build_model(context,data)
    recent_prices = data.history(context.security, 'price', context.lookback+1, '1d')
    # Get price changes
    price_changes = np.diff(recent_prices) > 0
        
    # Predict using our model and the recent prices
    prediction = context.model.predict(price_changes)
    
    objective = opt.MaximizeAlpha(
      context.output.sentiment_score
    )
    build_model(context,data)
    # part of constrains
    constrain_posTam = opt.PositionConcentration.with_equal_bounds(-1.5*context.max_posTam,context.max_posTam)
    max_lever = opt.MaxGrossExposure(context.max_lever)
    max_Volum = opt.MaxTurnover(context.max_Volum) 
    dollar_net = opt.DollarNeutral()
    
    
    #Risk Model Exposure
    factor_risk_constraints = opt.experimental.RiskModelExposure(
        context.risk_factor_betas,
        version=opt.Newest
    )

    order_optimal_portfolio(
        objective=prediction,
        constraints=[
            constrain_posTam,
            max_lever,
            max_Volum,
            factor_risk_constraints,
            dollar_net
        ]
    )

    pass